In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transform - same as your CIFAR10 one
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # match CIFAR-10 size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 4

# Use ImageFolder for your dataset
trainset = datasets.ImageFolder(root='cropped/', transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = datasets.ImageFolder(root='cropped/', transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Get class names
classes = trainset.classes
print("Classes:", classes)
